# Modify an existing slice
This notebook describes how to modify an existing slice.
Initially a slice containing two VMs (Node1, Node2) on a site1 connected with FABnetv4 service which is FABRIC's private IPv4 internet is provisioned.

It is then modified to add two VMs Node3 on site2 and Node4 on site1. Node3 is connected to FABNetv4 service and Node4 is then connected to Node2 via L2Bridge.

## Import the FABlib Library

In [ ]:
from fabrictestbed.slice_editor import ComponentModelType
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
fablib.show_config()

## Load the plugins

In [ ]:
from plugins import Plugins
import traceback
try:
    Plugins.load()
except Exception as e:
    traceback.print_exc()

## Create the Experiment Slice
Create two VMs Node1 and Node2 connected to NIC_Basic on site1.
Add a L3Network and connect Node1 to the L3Network.

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports)
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports)

In [ ]:
slice_name = 'MySlice-modify'
#[site1,site2] = fablib.get_random_sites(count=2)
#print(f"Sites: {site1}, {site2}")
site1 = "RENC"
site2 = "UKY"
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
node4_name = 'Node4'
network1_name = 'net1'
network2_name = 'net2'
network3_name = 'net3'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'
node4_nic_name = 'nic4'
iface2_name = None

### L2 Network

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

## Create the Slice
- Two VMs Node1 and Node2 on site1
- Node2 connected to L3 Network Service FABNetv4

In [ ]:
try:
    # Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]

    # Node2
    node2 = slice.add_node(name=node2_name, site=site1)
    iface2 = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    iface2_name = iface2.get_name()

    # Network
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv4')

    # Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Observe the Slice's Attributes

In [ ]:
### Print the slice
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
    for node in slice.get_nodes():
        print(f"{node}")
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## Modify Slice
- Add Node3 on site2 and Node4 on site1
- Add L3 Network connecting Node3 to it.
- Add L2Bridge Network connecting Node2 to Node4.

In [ ]:
print("Modifying the slice")
try:
    # Modify Slice
    slice = fablib.get_slice(name=slice_name)
    # Node3
    node3 = slice.add_node(name=node3_name, site=site2)
    iface3 = node3.add_component(model='NIC_Basic', name=node3_nic_name).get_interfaces()[0]

    # Node4
    node4 = slice.add_node(name=node4_name, site=site1)
    iface4 = node4.add_component(model='NIC_Basic', name=node4_nic_name).get_interfaces()[0]
    node2 = slice.get_node(name=node2_name)
    iface2 = node2.get_interface(name=iface2_name)

    # NetworkS
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface3], type='IPv4')
    net3 = slice.add_l2network(name=network3_name, interfaces=[iface2, iface4])

    # Submit Slice Request
    slice.modify()
    print("Slice modified successfully")
except Exception as e:
    traceback.print_exc()
    print(f"Exception: {e}")

## Observe the Slice's Attributes

In [ ]:
### Print the slice
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
    for node in slice.get_nodes():
        print(f"{node}")
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## Accept the modify

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.modify_accept()
except Exception as e:
    print(f"Exception: {e}")

## Configure IP Addresses

### Configure Node1 and Node3 (connected via FABNetv4)

In [ ]:
try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    network2 = slice.get_network(name=network2_name)
    network2_available_ips = network2.get_available_ips()
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_addr = network1_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())

    print("Node1 --- info-----------")
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    print (stdout)

    stdout, stderr = node1.execute(f'ip route list')
    print (stdout)
    node3 = slice.get_node(name=node3_name)
    node3_iface = node3.get_interface(network_name=network2_name)
    node3_addr = network2_available_ips.pop(0)
    node3_iface.ip_addr_add(addr=node3_addr, subnet=network2.get_subnet())
    node3.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    print("Node3 --- info-----------")
    stdout, stderr = node2.execute(f'ip addr show {node3_iface.get_os_interface()}')
    print(stdout)
    stdout, stderr = node2.execute(f'ip route list')
    print(stdout)
except Exception as e:
    print(f"Exception: {e}")

### Check connectivity between Node1 and Node3 over FABNetv4

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node3_addr}')
    print (stdout)
    print (stderr)

except Exception as e:
    print(f"Exception: {e}")

### Configure Node2 and Node4(connected via L2Bridge)

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)
    node2_iface = node2.get_interface(network_name=network3_name)
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print (stdout)
    node4 = slice.get_node(name=node4_name)
    node4_iface = node4.get_interface(network_name=network3_name)
    node4_addr = available_ips.pop(0)
    node4_iface.ip_addr_add(addr=node4_addr, subnet=subnet)
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    print(stdout)
except Exception as e:
    print(f"Exception: {e}")

### Check connectivity between Node2 and Node4 over L2Bridge

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)
    stdout, stderr = node2.execute(f'ping -c 5 {node4_addr}')
    print(stdout)
    print(stderr)

except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.


In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")